# Введение

В этом примере мы рассмотрим, как устроены и обучаются рекуррентные нейроные сети (Recurrent NN, RNN).
Рекуррентность -- это свойство некоторых слоёв или нейронов обращаться к собственным выходам как ко входам. 

Поскольку нейронная сеть вычисляет своё состояние в такт с вычислительными циклами, использование нейроном "предыдущего этапа" для вычисления текущего состояния эффективно реализует механизм памяти, который инженеры систем управления привыкли называть обратной связью в системе.

# Первые действия

Если при запуске ячеек блокнота возникают ошибки с описанием отсутствия некоторых библиотек на вашей машине, добавьте строки наподобие `!pip install sklearn` в следующую ячейку и выполните её.

Таким же образом из блокнота можно вызывать любые команды терминала (перемещать файлы, отправлять информацию через внешние порты).


In [ ]:
!pip install sklearn
!pip install torch
!pip install torchvision
!pip install textgenrnn

Рекуррентные сети позволяют моделировать зависимости, на развитие которых имеет влияние их же собственная предыстория. Вряд ли можно назвать применения RNN, которые невозможны на других ANN, в конце концов обратные связи всега можно "размотать" и получить плоскую модель без обратных связей.

![Каждый узел с обратной связью становится двумя узлами, один из которых обозначает прошлое состояние](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

Размотку даже используют, чтобы эффективно выстроить алгоритм обучения (оптисизации весов).

Поскольку RNN меняют свой состояние после вызова, и могут при одинаковых входах бесконечно долго возвращать различные выводы, их часто используют как обучаемыме генераторы с задавами вроде:
* сгенерировать много данных, похожих на выборку и обучить на них другую ANN (или сделать твит-бота который пишет новые тексты песен за некоторых артистов)
* моделировать функцию на основе ограниченного набора данных, в которой есть временна́я переменная (научить нейросеть [генерировать синусоиду](https://gist.github.com/spro/ef26915065225df65c1187562eca7ec4) или торговую динамику) в том случае, если вам **не хочется думать о структуре модели** (ARMAX? NARMAX? AR(3)? MA(7)?).

In [ ]:
# Import Libraries
#import torch
#import torch.nn as nn
#import torchvision.transforms as transforms
#from torch.autograd import Variable
#from sklearn.model_selection import train_test_split
from google.colab import files
from textgenrnn import textgenrnn
import os

В этой лабораторной рабоет мы будем пользоваться библиотекой `textgenrnn`, которая прячет от нас (абстрагирует нас от) всех промежуточных операций создания RNN для текстового корпуса.

Когда мы обучаем новую модель, `textgenrnn` позволяет указать размер и сложность нейросети с хорошим набором параметров:

In [ ]:
model_cfg = {
    'rnn_size': 128, # количество LSTM-нейронов на каждом уровне
    'rnn_layers': 4, # количество слоёв RNN
    'rnn_bidirectional': True, # включить обучение на тексте, прочитанном в обратную сторону (повышает качество обучения)
    'max_length': 40, # количество признаков (слов или букв), на базе которых предсказывается следующий признак
    'max_words': 10000, # наибольшее количество слов, котоыре будет знать сеть (только если признаки -- слова)
    'dim_embeddings': 100, # ...
    'word_level': False # если True, то мы хотим обучать модель на признаках-словах, но для этого требуется больший корпус и меньшее значение max_length
}

train_cfg = {
    'line_delimited': False, # Установить в True если каждая порция текста начинается с новой строки
    'num_epochs': 10, # чем больше, тем больше раз модель изучит корпус
    'gen_epochs': 2, # генерировать проверочный текст после некоторого количества эпох
    'batch_size': 1024, # ...
    'train_size': 0.8, # процент данных, которые мы будем использовать в обучении (<1.0 не позволяет модели переобучиться)
    'dropout': 0.8, # в каждую эпоху, игнорирует некоторую случайную часть входных данных, что позволяет модели лучше обобщить текст
    'max_gen_length': 300, # ...
    'validation': False, # если True, то после каждой эпохи будет проводиться валидация на остатке данных (обучение замедляется)
    'is_csv': False # ...
}

Пока, оставим эти параметры такими как есть. Запустите эти ячейки чтобы они сохранились в рабочей памяти. 

Мы дали время модулям загрузиться, теперь подумаем о наборе данных. У нас есть интересующие нас тексты в текстовых файлах, и загрузить их можно следующим образом:

In [ ]:
uploaded = files.upload()

Набор файлов будет храниться в папке, из которой вы запускаете данный блокнот.

Если файлы уже лежат на диске, просто не выполняйте эту ячейку.

Поскольку в файлах находятся несколько диссертаций в формате:

```
Название работы
Несколько строк автореферата
...
```

при загрузке можно было бы выбрасывать первую строку. Но пока можно прочитать список файлов таким образом:

In [ ]:
all_files = [n for n in uploaded]

# Создание и обучение

Создадим новую модель textgenrnn и начнём её обучать!

In [ ]:
model_name = 'colab'
textgen = textgenrnn(name=model_name)

#train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

texts = []
for f in uploaded:
  for line in open(f).readlines():
    texts.append(line)

print(len(texts))
  
textgen.reset()
textgen.train_new_model(texts,
    max_length=5,
    gen_epochs=5,
    num_epochs=10)

#train_function(
#    file_path=u'\n'.join(open(f).readlines() for f in uploaded),
#    new_model=True,
#    num_epochs=train_cfg['num_epochs'],
#    gen_epochs=train_cfg['gen_epochs'],
#    batch_size=1024,
#    train_size=train_cfg['train_size'],
#    dropout=train_cfg['dropout'],
#    validation=train_cfg['validation'],
#    is_csv=train_cfg['is_csv'],
#    rnn_layers=model_cfg['rnn_layers'],
#    rnn_size=model_cfg['rnn_size'],
#    rnn_bidirectional=model_cfg['rnn_bidirectional'],
#    max_length=model_cfg['max_length'],
#    dim_embeddings=100,
#    word_level=model_cfg['word_level'])

# Запустить генератор

При желании, от этого генератора можно получить весьма большое количество компилятивного текста.

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

Веса и конфигурацию сети можно скачать, чтобы использовать их в другой обстановке, например на RPi (см. бесплатный TTS движок festival):

In [ ]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

Чтобы воссоздать уже обученную сеть на собственном компьютере, запустите скрипт:

In [ ]:
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='colaboratory_weights.hdf5',
                       vocab_path='colaboratory_vocab.json',
                       config_path='colaboratory_config.json')

textgen.generate_samples(max_gen_length=1000)
textgen.generate_to_file('textgenrnn_texts.txt', max_gen_length=1000)

Colaboratory настроен против экспорта файлов больше 30 Mb, но всегда можно сохранить результат на Google-Drive, например.

# Вот и всё

Теперь в задании для вас есть несколько вопросов и предложенных модификаций.